In [1]:
import random


from ubii.interact.util import condition_property


class Test:
    def __init__(self):
        self._value = None

    @condition_property
    def a(self):
        print("normal get")
        return self._value

    @a.setter
    def a(self, value):
        print("normal set")
        self._value = value

x = Test()

In [2]:
import asyncio
async def wait_for_value():
    while True:
        value = await x.a.get()
        print(value)

async def set_value(range):
    for value in range:
        await x.a.set(value)
        await asyncio.sleep(int(4 * random.random()))

In [ ]:
await asyncio.gather(wait_for_value(), set_value(range=range(20)))

Creating Accessor
async get
async set
normal set
normal get
0
async get
async set
normal set
normal get
1
async get
async set
normal set
normal get
2
async get
async set
normal set
normal get
3
async get
async set
normal set
normal get
4
async get
async set
normal set
normal get
5
async get
async set
normal set
normal get
6
async get
async set
normal set
normal get
7
async get
async set
normal set
normal get
8
async get
async set
normal set
normal get
9
async get
async set
normal set
normal get
10
async get
async set
normal set
normal get
11
async get
async set
normal set
normal get
12
async get
async set
normal set
normal get
13
async get
async set
normal set
normal get
14
async get
async set
normal set
normal get
15
async get
async set
normal set
normal get
16
async get
async set
normal set
normal get
17
async get
async set
normal set
normal get
18
async get
async set
normal set
normal get
19
async get


In [ ]:
from itertools import chain, repeat


def _default_predicate():
    iter = chain(repeat(False, 1), repeat(True))
    print(iter)
    return iter.__next__

a = _default_predicate()
b = _default_predicate()

for i in range(5):
    print(a())
    print(b())